In [21]:
from src import frreg, obj1, obj2, tools
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Goal.1 : Partitioning $V_G$ and $V_S$

In [23]:
# [FR-reg]
df_relinfo = pd.read_csv("./test/test.info", sep='\t')
df_pheno = pd.read_csv("./test/test.phen", sep='\t')

# remove outliers
df_pheno = frreg.remove_outliers(df_pheno, "pheno")

# merge pheno with relatives
df_mrg = frreg.merge_pheno_info(df_pheno, df_relinfo)

# Partitioning Vg and Vs
df_frreg, msgs = frreg.familial_relationship_regression_DOR(
    df_mrg.drop(columns=["rcode", "relationship", "Erx"]),
    thred_pair=10,
    n_bootstrap=100
)

df_frreg 
# this is just example of FR-reg (recommend se < 0.01 for real analysis)

100%|██████████| 100/100 [00:00<00:00, 1186.01it/s]


,DOR,slope,se,p,n
0,1,0.306601,0.085488,0.000621,70
1,2,0.217332,0.110600,0.053560,68
2,3,0.396223,0.104670,0.000297,80


In [27]:
# [slope test]
df_frreg = pd.read_csv("test/test.DOR.frreg", sep='\t')
# resampling FR-reg to compute CIs
df_lmbds = obj1.resampleFrregCoefficients(df_frreg, n_resample=100)    

# slope test(FRLog-reg)
df_frlogreg = obj1.familialRelationshipLogRegression(df_lmbds)
sig = obj1._slopeSig(df_frlogreg["slope"])

# # save slope test results
print(sig)
(
    tools
    .raw2long(df_frlogreg, params=["slope", "intercept"])
)

High


,param,median,lower(2.5%),upper(97.5%)
0,slope,1.105632,1.029623,1.230799
1,intercept,-0.637859,-0.764133,-0.486668


In [29]:
df_frlogreg["slope"].quantile(0.025)

1.0296228661259403

In [30]:
# [prediction]
# resampling FR-reg to compute CIs
df_lmbds = obj1.resampleFrregCoefficients(df_frreg, n_resample=100)    

# save slope test results
df_gsw = obj1.prediction(
    df_lmbds, 
    [df_frlogreg["slope"].quantile(0.025), df_frlogreg["slope"].quantile(0.975)],
    print_prog=True
)

# save prediction results 
(
    tools
    .raw2long(df_gsw, params=df_gsw.columns)
)

1..2..3..4..5..6..7..8..9..10..

,param,median,lower(2.5%),upper(97.5%)
0,V(g),0.511882,0.468248,0.528538
1,V(s),0.043831,0.035727,0.063641
2,w,0.155000,0.010000,0.310000
3,mse_train,1.365692,1.239813,1.486774
4,mse_test,0.144835,0.063003,0.252460


# Goal.2 : Estimate $V_X$

In [33]:
# [FR-reg]
df_relinfo = pd.read_csv("./test/test.info", sep='\t')
df_pheno = pd.read_csv("./test/test.phen", sep='\t')

# remove outliers
df_pheno = frreg.remove_outliers(df_pheno, "pheno")

# merge pheno with relatives
df_mrg = frreg.merge_pheno_info(df_pheno, df_relinfo)

# Partitioning Vg and Vs
df_frreg, msgs = frreg.familial_relationship_regression_REL(
    df_mrg,
    thred_pair=10,
    n_bootstrap=100
)

df_frreg 
# this is just example of FR-reg (recommend se < 0.01 for real analysis)

100%|██████████| 100/100 [00:00<00:00, 975.11it/s]


,DOR,relationship,sex_type,Erx,slope,se,p,n
0,1,daughter-father,FM,0.707107,0.305082,0.107962,0.006163,70
1,1,daughter-mother,FF,0.500000,0.303681,0.089773,0.001186,70
2,1,daughter-sister,FF,0.750000,0.283295,0.114447,0.015771,70
3,1,different-sex-sibling,FM,0.353553,0.295412,0.109344,0.008675,70
4,1,son-brother,MM,0.500000,0.283899,0.098193,0.005127,70
5,1,son-father,MM,0.000000,0.307071,0.101472,0.003479,70
6,1,son-mother,FM,0.707107,0.292577,0.100556,0.004867,70
7,2,daughter-father-brother,FM,0.353553,0.209207,0.125096,0.099115,68
8,2,daughter-father-sister,FF,0.250000,0.215655,0.105094,0.044078,68
9,2,daughter-mother-sister,FF,0.375000,0.192082,0.121748,0.119345,68


In [34]:
# [regress out and prediction]
df_frreg = pd.read_csv("./test/test.REL.frreg", sep='\t')
df_x = obj2.estimateX(
    df_frreg,
    regout_bin=["DOR"],
    alpha_dicts={"type": "lambda", "weight":2},
    print_summary=True
)


              Prediction Result : 
              - Vx : 0.010(0.007,0.013)
              


In [36]:
# [predict Vx for male and female]
df_xmxf = obj2.estimateXmXfR(
    df_frreg, 
    regout_bin=["DOR", "sex_type"],
    alpha_dicts={"type": "lambda", "weight": 1},
    print_summary=True,
    )


              Prediction Result : 
              - Vx_male : 0.007(0.005,0.009)
              - Vx_female : 0.007(0.005,0.009)
              - r : 1.000(1.000,1.000)
              
